# OSIC Image Data Prep and Baseline Regression Model

 - This is really just meant to be a **demonstration**, and for various reasons (the runtime of image data prep, other ways the data is prepped and merged, using internet, etc.) this can't be used to make a submission.  If you're looking for a base script that is somewhat similar and allows for a submission, I would check out (https://www.kaggle.com/titericz/tabular-simple-eda-linear-model).
 - Some other general thoughts and findings:
     - I commented out the pickle data idea since the true holdout set is hidden, so we **can't pickle our submission data**.  I learned this the hard way.
     - For whatever reason, Kaggle doesn't like the way I merge some features to the submission file, so again I would check out (https://www.kaggle.com/titericz/tabular-simple-eda-linear-model) for base code for merging data for making submission.
     - First we model FVC, then we treat training confidence like hyperparameters and do random search. Next, we do a crude implementation of gradient descent without the gradient (so instead of a gradient telling us which direction to go, we try increasing and decreasing the value, see which does best, and take the better value).  Note that we also utilized hyperopt at one time, but this code was removed since it took too long to run (and it didn't really prove itself overly valuable in this case).  Finally we build a model for the confidence (because random search on new data is not efficient, stable, or flexible).  
     - In our model for confidence, we comment out code for Laplace Log Likelihood.  The tensorflow code works, but TF itself doesn't appear to fit both models at the same time.  I assume this is because both outputs are compared to confidence instead of 1 to fvc and 1 to confidence (but I don't know for sure).  In short, I couldn't get this to work the way I wanted, but I included the code if anyone wants to take a look.  
     - Finally we use the FVC model and Confidence model, to make our final predictions.
     - Again, this notebook is mainly a showcase of ideas, so take some of the code with a grain of salt.


### Other References

 - Feature Engineering ideas:  https://www.kaggle.com/yasufuminakama/osic-lgb-baseline?select=submission.csv

### Acknowledgements

https://www.kaggle.com/yasufuminakama

https://www.kaggle.com/titericz

# Imports & Settings 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [ ]:
!conda install -y gdcm -c conda-forge

In [ ]:
import pydicom
import math
import PIL
from PIL import Image
import numpy as np
from keras import layers
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
import tensorflow as tf
from tqdm import tqdm
%matplotlib inline

In [ ]:
from scipy.stats import shapiro
from scipy import stats 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pickle
import gc

### Configs

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

### Constants

In [ ]:
BATCH_SIZE = 30
TRAIN_VAL_RATIO = 0.35
EPOCHS_M1 = 200
EPOCHS_M2 = 400
LR = 0.005
imSize = 224

# Get Tabular Data

In [ ]:
train_df = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
test_df = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')
sub_df = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/sample_submission.csv')
print(train_df.shape)
print(test_df.shape)
train_df.head()

In [ ]:
sub_df['Patient'] = sub_df['Patient_Week'].apply(lambda x: x.split("_", 1)[0])
sub_df['Weeks'] = sub_df['Patient_Week'].apply(lambda x: x.split("_", 1)[1])
sub_df.head()

# Target Preprocessing

In [ ]:
plt.hist(train_df['FVC'])

In [ ]:
# https://machinelearningmastery.com/a-gentle-introduction-to-normality-tests-in-python/
# normality test
stat, p = shapiro(train_df['FVC'])
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.01
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')

### Transformed Target Distribution

In [ ]:
# transform training data & save lambda value 
_, fitted_lambda = stats.boxcox(train_df['FVC']) 
print("solved lambda: ", fitted_lambda)

In [ ]:
def BoxCoxTransform(x, lmbda):
    part1 = x**lmbda
    part2 = part1-1
    result = part2/lmbda
    return result

def ReverseBoxCoxTranform(x, lmbda):
    x = np.where(x<0,0,x)
    part1 = x*lmbda + 1
    result = part1**(1/lmbda)
    return result

In [ ]:
fitted_data = BoxCoxTransform(train_df['FVC'], lmbda=fitted_lambda)
plt.hist(fitted_data)

In [ ]:
# https://machinelearningmastery.com/a-gentle-introduction-to-normality-tests-in-python/
# normality test
stat, p = shapiro(fitted_data)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.01
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')

In [ ]:
# show that we can reverse transformed values back to original values
plt.hist(ReverseBoxCoxTranform(fitted_data, lmbda=fitted_lambda))

# Feature Preprocessing

### helper functions

In [ ]:
def preprocess_image(image_path, desired_size=imSize):
    im = pydicom.dcmread(image_path).pixel_array
    im = Image.fromarray(im, mode="L")
    im = im.resize((desired_size,desired_size)) 
    im = np.array(im).flatten().astype(np.uint8)
    return im

In [ ]:
def process_patient_images(patient_path, imSize=imSize):
    image_filenames = os.listdir(patient_path)
    final_array = np.zeros((imSize*imSize), dtype=np.uint8)
    total_images = len(image_filenames)
    for image_filename in image_filenames:
        image_path = patient_path + "/" + image_filename
        image_arr = preprocess_image(image_path, desired_size=imSize)
        final_array += image_arr
    final_array = final_array / total_images
    return final_array

### process training images

In [ ]:
# get the number of training images from the target\id dataset
N = train_df.shape[0]
# create an empty matrix for storing the images
x_train = np.empty((N, imSize*imSize), dtype=np.uint8)
# loop through the images from the images ids from the target\id dataset
# then grab the cooresponding image from disk, pre-process, and store in matrix in memory
for i, Patient in enumerate(tqdm(train_df['Patient'])):
    x_train[i, :] = process_patient_images(
        f'../input/osic-pulmonary-fibrosis-progression/train/{Patient}'
    )

### process test images

In [ ]:
# get the number of training images from the target\id dataset
N = test_df.shape[0]
# create an empty matrix for storing the images
x_test = np.empty((N, imSize*imSize), dtype=np.uint8)
# loop through the images from the images ids from the target\id dataset
# then grab the cooresponding image from disk, pre-process, and store in matrix in memory
for i, Patient in enumerate(tqdm(test_df['Patient'])):
    x_test[i, :] = process_patient_images(
        f'../input/osic-pulmonary-fibrosis-progression/train/{Patient}'
    )

### process submission images

In [ ]:
# get the number of training images from the target\id dataset
N = sub_df.shape[0]
# create an empty matrix for storing the images
x_sub = np.empty((N, imSize*imSize), dtype=np.uint8)
# loop through the images from the images ids from the target\id dataset
# then grab the cooresponding image from disk, pre-process, and store in matrix in memory
for i, Patient in enumerate(tqdm(sub_df['Patient'])):
    x_sub[i, :] = process_patient_images(
        f'../input/osic-pulmonary-fibrosis-progression/train/{Patient}'
    )

### process tabular features

When I went to make my own submission, I had some issues here.  First, pickling the data for later use was a major fail.  Also, for reasons I still don't understand, Kaggle doesn't like the way I merge my data for the submission file.  For merging features to the submission file, I would recommend checking out https://www.kaggle.com/titericz/tabular-simple-eda-linear-model and using their 'traintest' approach.  After pulling my hair out for a couple days, I found this to work.  Please, review this work for ideas, but don't use it as base code for a submission (or you'll go crazy).

In [ ]:
# one hot encoding
train_df['Sex_Male'] = train_df['Sex'].apply(lambda x: 1 if str(x)=='Male' else 0)
train_df['SmokingStatusEx'] = train_df['SmokingStatus'].apply(lambda x: 1 if str(x)=='Ex-smoker' else 0)
test_df['Sex_Male'] = test_df['Sex'].apply(lambda x: 1 if str(x)=='Male' else 0)
test_df['SmokingStatusEx'] = test_df['SmokingStatus'].apply(lambda x: 1 if str(x)=='Ex-smoker' else 0)

In [ ]:
train_df.head()

In [ ]:
# patient profile
patient_profile = train_df.groupby("Patient", as_index=False) \
                          .agg({'Percent':'mean', 'Sex_Male':'max', 'SmokingStatusEx':'max', 'Age':'mean', 'Weeks':'min'})
patient_profile = patient_profile.rename(columns={'Percent':'AvgPercent','Age':'AvgAge','Weeks':'BaseWeek'})

In [ ]:
patient_profile.head()

In [ ]:
# merge profile and create more features
train_df = train_df.merge(patient_profile[["Patient","AvgPercent","AvgAge","BaseWeek"]], on="Patient", how='left')
train_df['RelativeWeek'] = train_df['Weeks'].apply(lambda x: int(x)) - train_df['BaseWeek'].apply(lambda x: int(x))
test_df = test_df.merge(patient_profile[["Patient","AvgPercent","AvgAge","BaseWeek"]], on="Patient", how='left')
test_df['RelativeWeek'] = test_df['Weeks'].apply(lambda x: int(x)) - test_df['BaseWeek'].apply(lambda x: int(x))
sub_df = sub_df.merge(patient_profile, on="Patient", how='left')
sub_df['RelativeWeek'] = sub_df['Weeks'].apply(lambda x: int(x)) - sub_df['BaseWeek'].apply(lambda x: int(x))

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
sub_df.head()

In [ ]:
# final dataframes and tabular features
# train
x_train_tabular_features = train_df[['Weeks','AvgPercent','AvgAge','Sex_Male','SmokingStatusEx','BaseWeek','RelativeWeek']].values
# test
x_test_tabular_features = test_df[['Weeks','AvgPercent','AvgAge','Sex_Male','SmokingStatusEx','BaseWeek','RelativeWeek']].values
# submission
x_sub_tabular_features = sub_df[['Weeks','AvgPercent','AvgAge','Sex_Male','SmokingStatusEx','BaseWeek','RelativeWeek']].values

In [ ]:
# dimensionality reduction
pca = PCA(n_components=100)
pca.fit(x_train)

In [ ]:
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)
x_sub = pca.transform(x_sub)

In [ ]:
# merge
x_train_full = np.concatenate((x_train_tabular_features, x_train), axis=1)
x_test_full = np.concatenate((x_test_tabular_features, x_test), axis=1)
x_sub_full = np.concatenate((x_sub_tabular_features, x_sub), axis=1)

In [ ]:
scaler = StandardScaler()
scaler.fit(x_train_full)
x_train_full = scaler.transform(x_train_full)
x_test_full = scaler.transform(x_test_full)
x_sub_full = scaler.transform(x_sub_full)

### we add some squared features for some model flexability

Since we're building a regression model here, we have 2 options to account for flexability:
 - Include polynomial features
 - Bin our continuous variables and 1-hot encode the bins (leaving 1 group out)

For simplicity, and since our data is small and we don't want to overfit, I've just included 2nd order features.

In [ ]:
# squared features for some model flexability
x_train_full2 = np.square(x_train_full)
x_test_full2 = np.square(x_test_full)
x_sub_full2 = np.square(x_sub_full)

In [ ]:
# merge
x_train_full = np.concatenate((x_train_full, x_train_full2), axis=1)
x_test_full = np.concatenate((x_test_full, x_test_full2), axis=1)
x_sub_full = np.concatenate((x_sub_full, x_sub_full2), axis=1)

In [ ]:
print(x_train_full.shape)
print(x_test_full.shape)
print(x_sub_full.shape)

In [ ]:
# view data
x_train_full[:2,:]

In [ ]:
# This section is commented out, because it doesn't prove useful based on the way the holdout set is designed

# save the data to disk so we can save as a Kaggle Dataset
# and skip data preprocessing in other scripts
# filename = 'osic_processed_train_data_v1.pkl'
# pickle.dump(x_train_full, open(filename, 'wb'))
# filename = 'osic_processed_test_data_v1.pkl'
# pickle.dump(x_test_full, open(filename, 'wb'))
# filename = 'osic_processed_sub_data_v1.pkl'
# pickle.dump(x_sub_full, open(filename, 'wb'))

# Model FVC

In [ ]:
x_train_fvc, x_val_fvc, y_train_fvc, y_val_fvc = train_test_split(
    x_train_full, BoxCoxTransform(train_df['FVC'], lmbda=fitted_lambda),
    test_size=TRAIN_VAL_RATIO, 
    random_state=2020
)

In [ ]:
with strategy.scope():
    # define structure
    xin = tf.keras.layers.Input(shape=(x_train_full.shape[1], ))
    xout = tf.keras.layers.Dense(1, activation='linear')(xin)
    # put it together
    model1 = tf.keras.Model(inputs=xin, outputs=xout)
    # compile
    opt = tf.optimizers.RMSprop(LR)
    model1.compile(optimizer=opt, loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
# print summary
model1.summary()

In [ ]:
### define callbacks
earlystopper = EarlyStopping(
    monitor='val_mean_squared_error', 
    patience=30,
    verbose=1,
    mode='min'
)

lrreducer = ReduceLROnPlateau(
    monitor='val_mean_squared_error',
    factor=.5,
    patience=10,
    verbose=1,
    min_lr=1e-9
)

In [ ]:
print("Fit model on training data")
history = model1.fit(
    x_train_fvc,
    y_train_fvc,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS_M1,
    validation_data=(x_val_fvc, y_val_fvc),
    callbacks=[earlystopper,lrreducer]
)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
history_df[['mean_squared_error', 'val_mean_squared_error']].plot()

In [ ]:
train_df['FVC_pred1'] = ReverseBoxCoxTranform(model1.predict(x_train_full), lmbda=fitted_lambda)

In [ ]:
train_df.head()

In [ ]:
stats.describe(model1.predict(x_train_full))

# Random Search Confidence

To solve for confidence, we treat each row value as a hyperparameter and try some random search.  The values that produce the best score can be used to build a Confidence model.  I was able to use this in an actual submission.  

https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.randint.html

https://www.kaggle.com/yasufuminakama/osic-lgb-baseline

In [ ]:
%%time

best_confidence = np.zeros(len(train_df))
BestRandSearchScore = -1000000000
scorehist = []
print("running random search...")
for i in range(10000):
    trial_confidence = np.random.randint(70, 1000, size=len(train_df))
    train_df['Confidence'] = trial_confidence
    train_df['sigma_clipped'] = train_df['Confidence'].apply(lambda x: max(x, 70))
    train_df['diff'] = abs(train_df['FVC'] - train_df['FVC_pred1'])
    train_df['delta'] = train_df['diff'].apply(lambda x: min(x, 1000))
    train_df['score'] = -math.sqrt(2)*train_df['delta']/train_df['sigma_clipped'] - np.log(math.sqrt(2)*train_df['sigma_clipped'])
    score = train_df['score'].mean()
    if score>BestRandSearchScore:
        BestRandSearchScore = score
        best_confidence = trial_confidence
        print("best confidence values found in round {} with best score of {}".format(i,score))
    scorehist.append(BestRandSearchScore)

In [ ]:
plt.plot(scorehist)
plt.ylabel('best score')
plt.xlabel('round')
plt.show()

# Manual Descent

This is a crude implementation of gradient descent without the gradients (so instead of a gradient telling us which direction to go, we try increasing and decreasing the value, we see which direction give us a better score, and we take the value that gives us the better score).  Random Search is faster, but if we try to implement some kind of adaptive 'learning rate' (i.e. making big changes in the beginning and decreasing the changes as we go) this approach moves a bit faster.  I was able to use this in an actual submission.  

In [ ]:
def md_learning_rate(val):
    if val == 0:
        return 10
    elif val == 1:
        return 8
    else:
        return 5/np.log(val)

In [ ]:
%%time

Rounds = 100
best_md_confidence = np.zeros(len(train_df))
BestManualDescentScore = -1000000000
rowScore = -1000000000
train_df['Confidence'] = best_confidence
train_df['diff'] = abs(train_df['FVC'] - train_df['FVC_pred1']) # don't need to compute this every time
train_df['delta'] = train_df['diff'].apply(lambda x: min(x, 1000)) # don't need to compute this every time
for j in range(Rounds):
    for i in range(len(train_df)):
        originalValue = train_df['Confidence'].iloc[i]
        # try moving value up
        train_df['Confidence'].iloc[i] = originalValue + md_learning_rate(j)
        train_df['sigma_clipped'] = train_df['Confidence'].apply(lambda x: max(x, 70))
        train_df['score'] = -math.sqrt(2)*train_df['delta']/train_df['sigma_clipped'] - np.log(math.sqrt(2)*train_df['sigma_clipped'])
        scoreup = train_df['score'].mean()
        # try moving value down
        train_df['Confidence'].iloc[i] = originalValue - md_learning_rate(j)
        train_df['sigma_clipped'] = train_df['Confidence'].apply(lambda x: max(x, 70))
        train_df['score'] = -math.sqrt(2)*train_df['delta']/train_df['sigma_clipped'] - np.log(math.sqrt(2)*train_df['sigma_clipped'])
        scoredown = train_df['score'].mean()
        if scoreup>scoredown:
            train_df['Confidence'].iloc[i] = originalValue + md_learning_rate(j)
            rowScore = scoreup
        else:
            train_df['Confidence'].iloc[i] = originalValue - md_learning_rate(j)
            rowScore = scoredown
    if rowScore>BestManualDescentScore:
        BestManualDescentScore = rowScore
        best_md_confidence = train_df['Confidence'].to_numpy()
        if j % 10 == 0:
            print("best confidence values found in round {} with best score of {}".format(j,BestManualDescentScore))

In [ ]:
if BestManualDescentScore>BestRandSearchScore:
    best_confidence = best_md_confidence
    print("some manual descent improved confidence values")

# Model Confidence

Here we want to model confidence.  I tried adding Laplace Log Likelihood as an additional metric, but eventually removed it due to issues.  The metric works, but the model doesn't really solve for fvc and confidence at the same time, making the metric useless.  In hindsight I think I needed to change the way I was defining my model output, but I just dropped the idea for the time being.  I included the code in case anyone wanted to take a look.    

In [ ]:
# x_train, x_val, y_train1, y_val1, y_train2, y_val2 = train_test_split(
#     x_train_full, 
#     best_confidence,
#     BoxCoxTransform(train_df['FVC'], lmbda=fitted_lambda), 
#     test_size=TRAIN_VAL_RATIO, 
#     random_state=2020
# )

x_train, x_val, y_train, y_val = train_test_split(
    x_train_full, 
    best_confidence,
    test_size=TRAIN_VAL_RATIO, 
    random_state=2020
)

In [ ]:
# def Laplace_Log_Likelihood(y_true, y_pred):
#     # get predictions
#     y_pred1 = tf.cast(y_pred[:,0], dtype=tf.float32) # confidence
#     y_pred2 = tf.cast(y_pred[:,1], dtype=tf.float32) # fvc
#     # reverse box cox
#     tfz = tf.cast(tf.constant([0]), dtype=tf.float32) 
#     y_pred2 = tf.where(y_pred2<tfz,tfz,y_pred1)
#     lbda = tf.cast(tf.constant([0.376401998544658]), dtype=tf.float32) 
#     tf1 = tf.cast(tf.constant([1]), dtype=tf.float32)
#     p1 = tf.math.add(tf.math.multiply(y_pred2,lbda), tf1)
#     y_pred2 = tf.pow(p1,tf.math.divide(tf1,lbda)) # fvc reverse box cox
#     # laplace log likelihood                
#     threshold = tf.cast(tf.constant([70]), dtype=tf.float32) 
#     sig_clip = tf.math.maximum(y_pred1, threshold)
#     threshold = tf.cast(tf.constant([1000]), dtype=tf.float32) 
#     delta = tf.math.minimum(tf.math.abs(tf.math.subtract(y_true,y_pred2)),threshold)
#     sqrt2 = tf.cast(tf.constant([1.4142135623730951]), dtype=tf.float32) 
#     numerator = tf.math.multiply(sqrt2,delta)
#     part1 = tf.math.divide(numerator,sig_clip)
#     innerlog = tf.math.multiply(sqrt2,sig_clip)
#     metric = tf.math.subtract(-part1,tf.math.log(innerlog))
#     return tf.math.reduce_mean(metric)

In [ ]:
with strategy.scope():
    # define structure
    xin = tf.keras.layers.Input(shape=(x_train_full.shape[1], ))
    xout = tf.keras.layers.Dense(1, activation='linear')(xin)
    # put it together
    model2 = tf.keras.Model(inputs=xin, outputs=xout)
    # compile
    opt = tf.optimizers.RMSprop(LR)
    model2.compile(optimizer=opt, loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
# print summary
model2.summary()

In [ ]:
### define callbacks
earlystopper = EarlyStopping(
    monitor='val_mean_squared_error', 
    patience=3,
    verbose=1,
    mode='min'
)

lrreducer = ReduceLROnPlateau(
    monitor='val_mean_squared_error',
    factor=.5,
    patience=2,
    verbose=1,
    min_lr=1e-9
)

In [ ]:
print("Fit model on training data")
history = model2.fit(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS_M2,
    validation_data=(x_val, y_val),
    callbacks=[earlystopper,lrreducer]
)

In [ ]:
model2.save('model.h5')

In [ ]:
history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
history_df[['mean_squared_error', 'val_mean_squared_error']].plot()
# history_df[['Laplace_Log_Likelihood', 'val_Laplace_Log_Likelihood']].plot()

In [ ]:
model2.predict(x_test_full)

# Train Score

In [ ]:
fvc = model1.predict(x_train_full)
conf = model2.predict(x_train_full)[:,0]
fvc = ReverseBoxCoxTranform(fvc, lmbda=fitted_lambda)
train_df['FVC_pred1'] = fvc
train_df['Confidence'] = conf
train_df['sigma_clipped'] = train_df['Confidence'].apply(lambda x: max(x, 70))
train_df['diff'] = abs(train_df['FVC'] - train_df['FVC_pred1'])
train_df['delta'] = train_df['diff'].apply(lambda x: min(x, 1000))
train_df['score'] = -math.sqrt(2)*train_df['delta']/train_df['sigma_clipped'] - np.log(math.sqrt(2)*train_df['sigma_clipped'])
score = train_df['score'].mean()
print("train score: ", score)

In [ ]:
train_df.head()

# Test Score

In [ ]:
fvc = model1.predict(x_test_full)
conf = model2.predict(x_test_full)
fvc = ReverseBoxCoxTranform(fvc, lmbda=fitted_lambda)
test_df['FVC_pred1'] = fvc
test_df['Confidence'] = conf
test_df['sigma_clipped'] = test_df['Confidence'].apply(lambda x: max(x, 70))
test_df['diff'] = abs(test_df['FVC'] - test_df['FVC_pred1'])
test_df['delta'] = test_df['diff'].apply(lambda x: min(x, 1000))
test_df['score'] = -math.sqrt(2)*test_df['delta']/test_df['sigma_clipped'] - np.log(math.sqrt(2)*test_df['sigma_clipped'])
score = test_df['score'].mean()
print("test score: ", score)

In [ ]:
test_df.head()